# 02 Stats Basics for ML

Correlation, collinearity, bias/variance, and overfitting basics.


## Table of Contents
- [Correlation vs causation](#correlation-vs-causation)
- [Multicollinearity (VIF)](#multicollinearity-vif)
- [Bias/variance](#bias-variance)
- [Hypothesis testing](#hypothesis-testing)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Foundations notebooks build the intuition that prevents the most common mistakes in economic ML:
- leaking future information,
- evaluating with the wrong split strategy,
- over-interpreting coefficients.


## Prerequisites (Quick Self-Check)
- Comfort with basic Python + pandas (reading CSVs, making plots).
- Willingness to write short interpretation notes as you go.

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Using random splits on time series.
- Assuming correlation implies causation.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/00b_foundations/02_stats_basics_for_ml.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/00b_foundations/02_stats_basics_for_ml.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Concept
This notebook gives you the statistical vocabulary you'll use throughout the project.

You will build intuition for:
- when correlations are meaningful vs misleading,
- why coefficients can become unstable when features are correlated,
- how overfitting shows up as a gap between train and test performance,
- how to read hypothesis tests (p-values / confidence intervals) without over-trusting them.

## Distributions you will encounter in this project

Before diving in, here are the key probability distributions that appear in regression output and hypothesis tests:

- **Normal (Gaussian)**: The bell curve. Regression errors are often assumed approximately normal. The CLT says sample means become approximately normal as $n$ grows, even if the underlying data is not.
- **t-distribution**: Like the normal but with heavier tails. Used for coefficient t-tests when the sample is finite. Approaches the normal as $n \to \infty$. Degrees of freedom = $n - k - 1$ (observations minus parameters).
- **$\chi^2$ (chi-squared)**: Sum of squared standard normals. Appears in tests for heteroskedasticity (Breusch-Pagan, White) and goodness-of-fit.
- **F-distribution**: Ratio of two $\chi^2$ variables. Used for testing whether a group of coefficients are jointly zero (F-test). The `statsmodels` summary reports the overall F-statistic and its p-value.

You don't need to memorize the PDFs — just know when each distribution appears and what it tests.


## Primer: `statsmodels` vs `scikit-learn` (inference vs prediction)

This repo uses both libraries because they serve different goals:

- **Prediction (ML):** optimize out-of-sample accuracy → `scikit-learn`
- **Inference (econometrics):** interpret coefficients + quantify uncertainty → `statsmodels`

### Minimal `statsmodels` OLS pattern

```python
import statsmodels.api as sm

# X: DataFrame of features, y: Series target
Xc = sm.add_constant(X, has_constant="add")  # add intercept
res = sm.OLS(y, Xc).fit()
print(res.summary())
```

**Expected output / sanity check**
- a table with `coef`, `std err`, `t`, `P>|t|`, and a CI column
- coefficient names match your column names

### What you are looking at in `res.summary()`

- **coef**: $\\hat\\beta$ (estimated effect in the model)
- **std err**: estimated uncertainty $\\widehat{SE}(\\hat\\beta)$
- **t**: $\\hat\\beta / \\widehat{SE}(\\hat\\beta)$
- **P>|t|**: p-value for $H_0: \\beta=0$ (conditional on assumptions)
- **[0.025, 0.975]**: 95% confidence interval

### Robust standard errors (change uncertainty, not coefficients)

```python
# Cross-section heteroskedasticity
res_hc3 = res.get_robustcov_results(cov_type="HC3")

# Time series autocorrelation + heteroskedasticity
res_hac = res.get_robustcov_results(cov_type="HAC", cov_kwds={"maxlags": 4})
```

### Common pitfalls (and quick fixes)

- **Forgetting the intercept**
  - Fix: always `add_constant`.
- **Wrong SE for time series**
  - Fix: use HAC when residuals are autocorrelated.
- **Treating p-values as causal proof**
  - Fix: write the identification assumption; otherwise interpret as association.
- **Mixing prediction and inference**
  - Fix: use `sklearn` pipelines + time splits for prediction; use `statsmodels` for coefficient uncertainty.


<a id="correlation-vs-causation"></a>
## Correlation vs causation

### Goal
Simulate a classic confounding scenario where variables are correlated without a direct causal relationship.

### Why this matters in economics
Macro indicators often move together. If you interpret correlations as causal effects, you'll make confident but wrong stories.



### Your Turn (1): Simulate a confounder


In [ ]:
import numpy as np
import pandas as pd

# We will build: z -> x, z -> w, and z -> y.
# That makes x and y correlated even if x doesn't directly cause y.

rng = np.random.default_rng(0)
n = 800

# TODO: Simulate a hidden confounder z
z = ...

# TODO: Create x and w that both depend on z (plus noise)
x = ...
w = ...

# TODO: Create y that depends on z (plus noise)
y = ...

df = pd.DataFrame({'z': z, 'x': x, 'w': w, 'y': y})
df.head()



### Your Turn (2): Correlation matrix + interpretation


In [ ]:
# TODO: Compute df.corr() and interpret.
# Questions:
# 1) Are x and y correlated?
# 2) Does that mean x causes y?
# 3) Which variable is the common cause?
corr = df.corr(numeric_only=True)
corr



### Optional extension: a simple regression 'control' demo


In [ ]:
import statsmodels.api as sm

# TODO: Fit two regressions and compare the coefficient on x:
# 1) y ~ x
# 2) y ~ x + z
# Hint: sm.add_constant + sm.OLS
...



<a id="multicollinearity-vif"></a>
## Multicollinearity (VIF)

### Goal
Create highly correlated predictors and see how they affect coefficient stability.

### Key term
> **Definition:** **Multicollinearity** means your predictors are strongly correlated with each other.
It doesn't necessarily hurt prediction, but it can make coefficient interpretation unstable.



### Your Turn (1): Build correlated features


In [ ]:
from src.econometrics import vif_table

# TODO: Create x1 and x2 that are almost the same
rng = np.random.default_rng(1)
n = 600
x1 = rng.normal(size=n)
x2 = ...  # make this highly correlated with x1

# Target depends mostly on x1
eps = rng.normal(scale=1.0, size=n)
y2 = 1.0 + 2.0 * x1 + eps

df2 = pd.DataFrame({'y': y2, 'x1': x1, 'x2': x2})
df2[['x1','x2']].corr()



### Your Turn (2): Compute VIF + interpret


In [ ]:
# TODO: Compute VIF for x1 and x2.
# How large are the VIFs? What does that suggest?
vif_table(df2, ['x1', 'x2'])



### Your Turn (3): Fit a regression and inspect coefficient stability


In [ ]:
import statsmodels.api as sm

# Fit y ~ x1 + x2 and inspect coefficients.
# TODO: Compare to fitting y ~ x1 alone.
X_both = sm.add_constant(df2[['x1', 'x2']])
res_both = sm.OLS(df2['y'], X_both).fit()

X_one = sm.add_constant(df2[['x1']])
res_one = sm.OLS(df2['y'], X_one).fit()

# TODO: Print the two coefficient estimates on x1
...



<a id="bias-variance"></a>
## Bias/variance

### Goal
See overfitting as a train/test gap by comparing a simple model vs a flexible one.

### Key term
> **Definition:** **Overfitting** happens when a model fits noise in the training data and fails to generalize.



### Your Turn (1): Create a non-linear dataset


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

rng = np.random.default_rng(2)
n = 400

# TODO: Create x on [-3, 3]
x = ...

# TODO: Create y = sin(x) + noise (nonlinear)
y = ...

# Train/test split (random is OK here because this is NOT time series)
split = int(n * 0.8)
X = x.reshape(-1, 1)
X_tr, X_te = X[:split], X[split:]
y_tr, y_te = y[:split], y[split:]

...



### Your Turn (2): Fit linear vs tree and compare errors


In [ ]:
# TODO: Fit LinearRegression and a DecisionTreeRegressor
# Compute RMSE on train and test for both
...



### Your Turn (3): Control model complexity


In [ ]:
# TODO: Refit the tree with different max_depth values (e.g., 2, 4, 8, None)
# Track train/test RMSE and describe the pattern.
...



## Primer: Hypothesis testing (p-values, t-stats, confidence intervals)

You will see p-values, t-statistics, and confidence intervals in regression output (especially `statsmodels`). This primer gives you the minimum to interpret them correctly.

### The objects (plain language)

- **Null hypothesis** $H_0$: the default claim (often “no effect”).
- **Alternative** $H_1$: the claim you consider if the data looks inconsistent with $H_0$.
- **Test statistic**: “how far” your estimate is from the null, in uncertainty units.
- **p-value**: probability (under the null *and model assumptions*) of seeing a test statistic at least as extreme as observed.
- **Confidence interval (CI)**: a range of parameter values consistent with the data under assumptions.

### What a p-value is NOT

- Not the probability $H_0$ is true.
- Not the probability the model is correct.
- Not a measure of economic importance.

### Regression t-test intuition

In OLS, a common test is $H_0: \\beta_j = 0$.

$$
t_j = \\frac{\\hat\\beta_j}{\\widehat{SE}(\\hat\\beta_j)}
$$

If you change your SE estimator (HC3/HAC/cluster), you change $\\widehat{SE}$ and therefore the p-value, even if the coefficient stays the same.

### Expected output / what you should look at in `res.summary()`

- `coef`: effect size (in model units)
- `std err`: uncertainty
- CI columns: magnitude + uncertainty together

### Common pitfalls in this project

- Macro time series often have autocorrelation → naive SE too small → use HAC when interpreting p-values.
- Multiple testing/spec-search can produce small p-values by chance.
- Predictive success ≠ causal interpretation.

### Tiny demo (toy; not project data)

```python
import numpy as np
import pandas as pd
import statsmodels.api as sm

rng = np.random.default_rng(0)
n = 300
x = rng.normal(size=n)
y = 1.0 + 0.5 * x + rng.normal(scale=1.0, size=n)

df = pd.DataFrame({"y": y, "x": x})
X = sm.add_constant(df[["x"]])
res = sm.OLS(df["y"], X).fit()
print(res.summary())
```


<a id="hypothesis-testing"></a>
## Hypothesis testing

### Goal
Make p-values and confidence intervals concrete with a toy example.

### Your Turn (1): One-sample t-test
Simulate a sample whose true mean is not 0 and test whether you can detect it.



In [ ]:
from scipy import stats

rng = np.random.default_rng(3)

# TODO: Simulate x with a small non-zero mean (e.g., 0.1) and some noise
x = ...

# TODO: Run a one-sample t-test for mean == 0
t_stat, p_val = ...
print('t:', t_stat, 'p:', p_val)

# TODO: Explain: what does this p-value mean in words?
...



### Your Turn (2): Regression coefficient test


In [ ]:
import statsmodels.api as sm

# TODO: Simulate a simple linear relationship y = 1 + 0.5*x + noise
rng = np.random.default_rng(4)
n = 300
x = ...
y = ...

df_ht = pd.DataFrame({'y': y, 'x': x})
X = sm.add_constant(df_ht[['x']])
res = sm.OLS(df_ht['y'], X).fit()

# TODO: Print coefficient, SE, p-value, and 95% CI for x
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Run a quick sanity check on any DataFrame/Series you created in this notebook.
# Example (adjust variable names):
# assert df.index.is_monotonic_increasing
# assert df.isna().sum().sum() == 0
#
# TODO: Write 2-3 sentences:
# - What would leakage look like in YOUR code?
# - How would you detect it?
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Correlation vs causation</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_stats_basics_for_ml — Correlation vs causation
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
n = 800
# Hidden confounder
z = rng.normal(size=n)

# Two observed variables both driven by z
x = z + rng.normal(scale=0.8, size=n)
w = z + rng.normal(scale=0.8, size=n)

# Target driven by z (not by x directly)
y = 2.0 * z + rng.normal(scale=1.0, size=n)

df = pd.DataFrame({'z': z, 'x': x, 'w': w, 'y': y})
df.corr(numeric_only=True)
```

</details>

<details><summary>Solution: Multicollinearity (VIF)</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_stats_basics_for_ml — Multicollinearity (VIF)
import statsmodels.api as sm
from src.econometrics import vif_table

# Build highly correlated predictors
rng = np.random.default_rng(1)
n = 600
x1 = rng.normal(size=n)
x2 = 0.95 * x1 + rng.normal(scale=0.2, size=n)
y = 1.0 + 2.0 * x1 + rng.normal(scale=1.0, size=n)
df = pd.DataFrame({'y': y, 'x1': x1, 'x2': x2})

vif_table(df, ['x1', 'x2'])

X = sm.add_constant(df[['x1', 'x2']])
res = sm.OLS(df['y'], X).fit()
print(res.summary())
```

</details>

<details><summary>Solution: Bias/variance</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_stats_basics_for_ml — Bias/variance
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Simple 1D regression problem
rng = np.random.default_rng(2)
n = 400
x = np.linspace(-3, 3, n)
y = np.sin(x) + rng.normal(scale=0.2, size=n)

X = x.reshape(-1, 1)
split = int(n * 0.8)

lin = LinearRegression().fit(X[:split], y[:split])
tree = DecisionTreeRegressor(random_state=0).fit(X[:split], y[:split])

rmse_lin_tr = mean_squared_error(y[:split], lin.predict(X[:split]), squared=False)
rmse_lin_te = mean_squared_error(y[split:], lin.predict(X[split:]), squared=False)
rmse_tree_tr = mean_squared_error(y[:split], tree.predict(X[:split]), squared=False)
rmse_tree_te = mean_squared_error(y[split:], tree.predict(X[split:]), squared=False)

(
    {'linear_train': rmse_lin_tr, 'linear_test': rmse_lin_te},
    {'tree_train': rmse_tree_tr, 'tree_test': rmse_tree_te},
)
```

</details>

<details><summary>Solution: Hypothesis testing</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_stats_basics_for_ml — Hypothesis testing
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

# One-sample t-test: mean(x) == 0?
rng = np.random.default_rng(3)
x = rng.normal(loc=0.1, scale=1.0, size=200)
t_stat, p_val = stats.ttest_1samp(x, popmean=0.0)
print('t:', t_stat, 'p:', p_val)

# Regression coefficient test: slope == 0?
rng = np.random.default_rng(4)
n = 300
x2 = rng.normal(size=n)
y2 = 1.0 + 0.5 * x2 + rng.normal(scale=1.0, size=n)

df = pd.DataFrame({'y': y2, 'x': x2})
X = sm.add_constant(df[['x']], has_constant='add')
res = sm.OLS(df['y'], X).fit()

print(res.summary())
```

</details>

